# 循环神经网络 Recurrent Neural Networks

## 1. 为什么要使用序列模型 Why sequence models

![Examples of sequence data.png](img/Examples of sequence data.png)

## 2. 标记符号 Notations

以命名实体识别（Named Entity Recognition）问题为例，我要需要一套标记符号，来描述这个监督学习问题。这里，用符号 $x^{(i)<t>}$ 表示第 i 个训练集的第 t 个词，用符号 $T_x^{(i)}$ 表示第 i 个训练集总共的单词数。目标变量 y 也类似。

![Motivating example RNN.png](img/Motivating example RNN.png)

由于这里是一个NLP（Natural Language Processing）问题，就涉及到如何来表示单词。通常我们会构造一个词典，以30000~50000的单词量为最常见，也有100000级别的词典，甚至在一些互联网公司，百万级别的词典也不是没有。有了词典之后，每个单词就会有一个序号。而将所有单词进行One-Hot编码，就可以获得表示这个单词的向量。

![Representing words.png](img/Representing words.png)

## 3. 循环神经网络模型 Recurrent Neural Network Model

使用标准神经网络模型来处理序列数据，主要会面临以下两个问题：1）序列数据的输入输出长度，对于不同的样本，可以是不同的；2）标准神经网络不会共享对序列数据的位置相关特征。

![Why not a standard network.png](img/Why not a standard network.png)

循环神经网络的模型：初始化 $a^{<0>}$ 为0向量，红色的参数共享，序列中每个单词的激活结果，作为下一个单词模型的输入之一。下图的左右描述是等价的。注意到这个RNN模型实际上是单向RNN，顺序在前的单词可以对顺序在后的单词施加影响，反之则不行，因而模型也就无法区分下面两个句子的不同。后面会说，经过一点简单的修改，也可以获得**双向RNN（Bidirected RNN）**

![Recurrent Neural Networks.png](img/Recurrent Neural Networks.png)

RNN的前向传播过程：

![Forward Propagation.png](img/Forward Propagation.png)

部分参数可以合并，从而简化计算：

![Simplified RNN notation.png](img/Simplified RNN notation.png)

## 4. 通过时间的反向传播 Backpropagation through time

![Backpropagation through time.png](img/Backpropagation through time.png)

## 5. 不同类型的循环神经网络 Different types of RNNs

上面提到的循环神经网络，都属于 $T_x = T_y$ 的多对多的循环神经网络，适用于命名实体识别问题。另外，也存在着其它不同类型的循环神经网络。

比如对于情感分析问题，可以构造多对一的RNN，甚至一对一，输出可以是二分类，也可以是1星到5星的多分类；而对于音乐生产问题，则可以构造一对多的RNN，其中输入可能是音乐类型，输出是音乐的序列数据；对于机器翻译问题，可以构造另一种输入输出的长度不同的RNN。

![Summary of RNN types.png](img/Summary of RNN types.png)

## 6. 语言模型和序列生成 Language Model and sequence generation

语言模型是自然语言处理的一个基础性而重要的任务，而RNN可以有效地进行语言模型建模。

以语音识别为例，下面两个句子的读音是完全一致的，语言模型可以给出每个句子的概率，通过这个概率帮助语音识别系统判断应该输出哪个句子。语言模型也可以用来帮助机器翻译。

![What is language modelling.png](img/What is language modelling.png)

建立语言模型，首先需要有一个**语料库 corpus**作为训练集，所谓语料库就是特定语言的一大批句子。第二部是**Tokenize**，也就是建立词典，并将语料库中的单词转换为向量的过程。注意到一般词典中会包含两个特殊的词，< EOS >表示句子结束，而< UNK >表示未知的词。

![Language modelling with an RNN.png](img/Language modelling with an RNN.png)

RNN实现的语言模型，令 $x^{<t>}=y^{<t-1>}$。$\hat{y}^{<1>}$ 输出的是第一个词的概率分布，是一个对字典中所有词的softmax回归。而 $\hat{y}^{<2>}$ 输出的是给定第一个词时，第二个词取值的条件概率分布。以此类推。

而上面提到的句子的概率，就是所有词概率的乘积，如右下角所示。

![RNN model.png](img/RNN model.png)

## 7. 生成新序列 Sampling novel sequences

有了训练完成的语言模型后，我们可以根据这个模型给定的多项分布以及后续的条件概率多项分布，来生成新的句子。如果词典中包含了 < EOS > 字符，则可以在生成该字符后中止句子。而对于 < UNK > 字符，如果不希望生成的句子里有未知字符，则拒绝该字符，继续生成即可。

![Sampling a sequence from a trained RNN.png](img/Sampling a sequence from a trained RNN.png)

上面介绍的语言模型，是单词级别的。也可以构造字符级别的语言模型。字符级别的语言模型，就不需要额外处理未知的单词了，但其计算量更大，对于生成长序列时表现也不够好，目前只有在少数情况下会使用字符级别的语言模型。

![Character-level language model.png](img/Character-level language model.png)

## 8. 循环神经网络的梯度消失问题 Vanishing gradients with RNNs

下面列举了两个句子，根据主语的单复数（cat or cats），跟在一大段定语从句后面的be动词会需要不同（was or were）。而目前提到的RNN，处理这个**长依赖 long-range dependencies**的能力不强，究其原因，可以归结于梯度消失。RNN中每个单词，都相当于神经网络的一层。当RNN的网络非常深时，很难使得反向传播比较强地影响到最初几层。为了应对RNN的梯度消失问题，有一系列的解决方案。以GRU和LSTM为主要代表。

RNN的梯度爆炸：只需要进行**梯度裁剪gradient cliping**即可，相对好解决。

![Vanishing gradients with RNNs.png](img/Vanishing gradients with RNNs.png)

## 9. GRU单元 Gated Recurrent Unit

![RNN unit.png](img/RNN unit.png)

![GRU simplified.png](img/GRU simplified.png)

![Full GRU.png](img/Full GRU.png)

## 10. 长短期记忆单元 Long Short Term Memory

LSTM是更复杂的模型，使用也更为广泛。GRU的计算量会小一些。

![LSTM in pictures.png](img/LSTM in pictures.png)

## 11. 双向循环神经网络 Bidirectional RNN

BRNN和LSTM几乎是NLP问题中的标配。

缺点：需要完整的序列数据来预测结果，而无法仅使用截止到当前的序列数据。对于实时要求高的系统，复杂度更高。

![Bidirectional RNN.png](img/Bidirectional RNN.png)